In [1]:
from utils import *

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

config = get_config('config.yaml')

[nltk_data] Downloading package stopwords to /Users/enlik/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# df2 = read_csv_from_gdrive(config['csv_input']['bolt_apple'])
# total_reviews = len(df2)
# unique_users  = ddlen(df2['userName'].unique())
# mean = df2['rating'].mean()

# print(f'Total English reviews: {total_reviews} \n')
# print(f'Total unique users : {unique_users}')
# print(f'Total users who gave multiple reviews: {total_reviews - unique_users}\n')
# print(f'Average rating for this app based on the textual reviews: {round(mean,2)} \n')

# NLP based Preprocessing

In [16]:
%run ./AR_Miner/AR_util.py
%run ./AR_Miner/AR_reviewInstance.py

# Inputs:
datasetName = "swiftkey" # four apps: facebook, templerun2, swiftkey, tapfish:
# datasetName = "templerun2" # four apps: facebook, templerun2, swiftkey, tapfish
rmStopWords = True # Removing stop words lead to information loss and bad f-score
rmRareWords = True # Remove the word with low frequency
skParse = False # set skParse True to directly read of the data that has been filtered out

# Outputs:
if(skParse == False):
    trainSet, testSet, unlabelSet, vocabulary = AR_parse(datasetName, rmStopWords, rmRareWords)

print('\n')
# trainSet[0].printSelf()

Vocabulary size for swiftkey : 1706
Training set Size: 1000
Testing set Size: 2000
Unlabeling set Size: 3282




# Naive Bayes based Filtering

In [19]:
%run ./AR_Miner/AR_classifier.py
import time
start_time = time.clock()

useSVM = True # SVM is way better than EMNB in the testing
if(skParse == False):
    if(useSVM == False):
#         informRev, informMat = AR_emnb(trainSet, testSet, unlabelSet, vocabulary, datasetName)
        informRev, uninformRev, informMat = AR_emnb(trainSet, testSet, unlabelSet, vocabulary, datasetName)
    else:
#         informRev, informMat = AR_svm(trainSet, testSet, unlabelSet, vocabulary, datasetName)
        informRev, uninformRev, informMat = AR_svm(trainSet, testSet, unlabelSet, vocabulary, datasetName)
    print(time.clock() - start_time, "seconds")
    # write the result back to the file (optional)
    # AR_writeReviews(informRev, datasetName)
    
else:
    # directly read from the file
    informRev, informMat, vocabulary = AR_loadReviews(datasetName)

print("Number of informative reviews: " + str(len(informRev)))
print("Number of uninformative reviews: " + str(len(uninformRev)))
print('\n')
informRev[1].printSelf()
print('\n')
uninformRev[5].printSelf()

No more progress, stopping EM at iteration 50
Average F-Score for the test data: 0.788007420330797
71.22413099999999 seconds
Number of informative reviews: 1061
Number of uninformative reviews: 2221


Review id: 3003 Rating: 1 Content: crash chrome everi minut Ntokens: 4 TS:  Group:  Prob: 0.7745828828324669 label: 1
Raw text: crashes chrome every 5 minutes


Review id: 3007 Rating: 5 Content: love product Ntokens: 2 TS:  Group:  Prob: 0 label: 0
Raw text: love this product


In [22]:
uninformRev[90].printSelf()

Review id: 3137 Rating: 5 Content: simpli brilliant Ntokens: 2 TS:  Group:  Prob: 0 label: 0
Raw text: simply brilliant


# LDA topic clustering

In [5]:
# %run ./AR_Miner/AR_lda.py

# n_topics = 20 # the number of topics
# doc_topic, vocab, top_words_list = AR_lda(informMat, vocabulary, n_topics)

# Ranking Algorithms for Importance

In [6]:
# %run ./AR_Miner/AR_ranker.py

# wg = [0.85, 0.15]
# group_scores, sorted_group_indices = group_rank(doc_topic, wg, informRev)
# print('Group scores:\n' + str(group_scores) + '\n')
# print('Group in order of importance:\n' + str(sorted_group_indices))

In [7]:
# %run ./AR_Miner/AR_textrank.py

# AR_tfIdf(informRev)
# rankrevText = AR_textrank(doc_topic, informRev)

In [8]:
# # print the top two reviews of the first five groups:
# for i in range(5): # number of groups to print
#     print("Instance review for topic group: " + str(i))
#     for j in range(2): # number of top reviews to print
#         r_ind = rankrevText[i][j][0]
#         score = rankrevText[i][j][1]
#         print(str(j+1) + "th review " + "Text: " +  informRev[r_ind].text + " Score: " + str(score))
#     print ("\n")    

In [9]:
# %run ./AR_Miner/AR_reviewRanking.py

# AR_tfIdf(informRev)
# weight = [0.25, 0.25, 0.25, 0.25]
# rankrevTopic = instance_ranking(doc_topic, weight, informRev)

# Visualization

In [10]:
# %run ./AR_Miner/AR_visualization.py

# group_count = 10
# plot_group_ranking(group_scores, sorted_group_indices, top_words_list, group_count)
# print("Ranking all the reviews in the first group via text rank")
# plot_instance_ranking(sorted_group_indices[0], informRev, rankrevText, 10)
# print("Ranking all the reviews in the first group via topic modeling")
# plot_instance_ranking(sorted_group_indices[0], informRev, rankrevTopic, 10)